<a href="https://colab.research.google.com/github/LubnaM/GANS/blob/main/GANs_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:

!pip install tensorflow ipywidgets tensorflow-datasets tensorflow-gpu

In [ ]:
#Bringing tensorflow fashion_mnist dataset
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

ds=tfds.load('fashion_mnist', split='train')

In [ ]:
ds.as_numpy_iterator().next()['image'] #replace image by label to view the label

In [ ]:
import numpy as np
dataiterator=ds.as_numpy_iterator()


In [ ]:
batch=dataiterator.next() # in batch, image by image will be called if we call .next()


In [ ]:
#visualization of the dataset
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx in range(4):
    sample=dataiterator.next() #sample here have 2 features image and label
    # This to plot image in to a sub plot, squeeze is to remove the 3rd dimention
    ax[idx].imshow(np.squeeze(sample['image']))
    # add image label as a title
    ax[idx].title.set_text(sample['label'])

In [ ]:
' Dataset preprocessing '
# Scaling the features to be between 0 and 1, instead of 1 to 255 ...
def scale_images(data):
  image=data['image']
  return image/255

In [ ]:
'--- Build the neural network ---'
#Conditional GAN is what we need to implement, we need to pass what type of text we want to generate. See this utube
#A data pipeline for building a tensorflow: cache > shuffle > batch > prefetch
ds=tfds.load('fashion_mnist', split='train')
#scale the image thru image function
ds=ds.map(scale_images)
#store intermediate results in memory or on disk. This can significantly speed up subsequent operations by avoiding the need to recompute results
ds=ds.cache()
ds=ds.shuffle(60000)
#Batch in to 128 images samples
ds=ds.batch(128)
#Prefetch reduces the bottlenecking
ds=ds.prefetch(64)

ds.as_numpy_iterator().next().shape


In [ ]:
' now  generator will try to generate an image fashion, and discriminator will try to spot the fake image'
#Sequential api for the generator and discriminator
from tensorflow.keras.models import Sequential
#bring the layers for the neuralnetwork. LeakyRely
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Reshape, LeakyReLU, UpSampling2D, BatchNormalization


In [ ]:
def build_generator():
  model=Sequential()
  #what number of input to take
  #128 passed to generator of spatial latent dimentions 7 X 7 X128 hidden layer
  'takes in random value, and reshape it by 7*7*128, which is a begining of a generated image'
  model.add(Dense(7*7*128, input_dim=128)) # For generator this is a random input of 128 vector size, which we will use it to generate the image randomly.`
  # After a series of convolutional and pooling layers, you might end up with an output that has the shape of (7, 7, 128).
  # You would then flatten this output (which has 7 * 7 * 128 = 6272 elements) and pass it to a dense layer to combine these features into higher-level representations.
  model.add(LeakyReLU(0.2)) # to cater for non linearity
  model.add(Reshape((7,7,128))) # reshape will take a dense of output of 6272 and convert is in to a begining of an image

  'upsampling block1'
  model.add(UpSampling2D()) #
  model.add(Conv2D(128, kernel_size=5, padding='same'))
  model.add(LeakyReLU(0.2))

  'upsampling block2'
  model.add(UpSampling2D()) #
  model.add(Conv2D(128, kernel_size=5, padding='same'))
  model.add(LeakyReLU(0.2))

  'Conv block1'
  model.add(Conv2D(128, kernel_size=4, padding='same'))
  model.add(LeakyReLU(0.2))

  'Conv block2'
  model.add(Conv2D(128, kernel_size=4, padding='same'))
  model.add(LeakyReLU(0.2))

  'conv layer to get 1 output channel'
  model.add(Conv2D(1, kernel_size=4, padding='same', activation='sigmoid'))

  return model # only block 1 can be sufecient but adding more sofistication will allow the generator to learn


In [ ]:
generator=build_generator()
generator.summary()

In [ ]:
img=generator.predict(np.random.rand(4,128,1))# 4 images


In [ ]:
'Generate new images and test generator'
#img
#visualize it
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(img):
    ax[idx].imshow(np.squeeze(img))
    ax[idx].title.set_text(idx)

In [ ]:
'-- Build the discriminator -- '
def build_discriminator():
  model=Sequential()
  #first conv block
  model.add(Conv2D(32, kernel_size=5, input_shape=(28,28,1))) #(28*28 will be pashed in to 32  filter with shape 5*5 , codensing down the information its getting)
  model.add(LeakyReLU(0.2)) # recomended practice when building GANs
  model.add(Dropout(0.4)) # for regularization
  #second conv block
  model.add(Conv2D(64, kernel_size=5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  #second conv block
  model.add(Conv2D(64, kernel_size=5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  #Third conv block
  model.add(Conv2D(128, kernel_size=5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  #Fourth conv block
  model.add(Conv2D(256, kernel_size=5))
  model.add(LeakyReLU(0.2))
  model.add(Dropout(0.4))
  #Flatten then pass to dense layer
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid')) # 1 is false image, and 0 is true image.

  return model

In [ ]:
discriminator=build_discriminator()


In [ ]:
discriminator.summary()

In [ ]:
#test the generated images
#discriminator.predict(img) # if 1 its a fake if less, its nearly near the truth value
# Add a batch dimension to the image data
discriminator.predict(np.expand_dims(img, axis=0)) # if 1 its a fake if less, its nearly near the truth value

In [ ]:
' - custom training loop to train the generator and discriminator simultanuesly -'
#setup some losses and optimizes
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
g_opt = Adam(learning_rate=0.0001) #Generator faster
d_opt = Adam(learning_rate=0.00001) #discriminator we dont want discriminator to go too fast than generator
loss_fn = BinaryCrossentropy()

In [ ]:
import tensorflow as tf
tf.concat([tf.zeros_like(tf.random.normal((6,128))),tf.ones_like(tf.random.normal((6,128)))],axis=0) # This is  will be generating a bunch of ones in to the generator and g

In [ ]:
#importing the basemodel class to subclass our training step
from tensorflow.keras.models import Model
class FashionGAN(Model):
  def __init__(self, generator, discriminator, *args,**kwargs):
    # pass args and kwars to base class
    super().__init__(*args,**kwargs)
    #create attributes for generators and disc
    self.generator=generator
    self.discriminator=discriminator

  def compile(self,g_opt,d_opt,loss_fn,*args,**kwargs):
    super().compile(*args,**kwargs)
    self.g_opt=g_opt
    self.d_opt=d_opt
    self.loss_fn=loss_fn

  def train_step(self,batch): # batch can 128 images *28 * 28 *1)
    #get the data
    real_images=batch
    fake_images=self.generator(tf.random.normal((128,128,1)), training = False) # False means generator is not training at the moment its just making predictions

    #Train the discriminator
    with tf.GradientTape() as d_tape:
      #pass the real and fake model in to the discriminator model.
      yhat_real=self.discriminator(real_images, training=True)
      yhat_fake=self.discriminator(fake_images, training=True)
      yhat_realfake=tf.concat([yhat_real, yhat_fake], axis=0)
      #create labels, this below will assign the labels below are true y_realfake , a labels
      y_realfake=tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)

      #add noise to the TRUE output to speedup learning
      noise_real = 0.15 * tf.random.uniform(tf.shape(yhat_real))
      noise_fake = -0.15 * tf.random.uniform(tf.shape(yhat_fake))
      y_realfake += tf.concat([noise_real, noise_fake],axis=0)

      #calsulate loss
      total_d_loss=self.loss_fn(y_realfake, yhat_realfake)
    #apply back propagation - nnlearn
    d_gradient=d_tape.gradient(total_d_loss, self.discriminator.trainable_variables)
    self.d_opt.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))


    #Train the generator
    with tf.GradientTape() as g_tape:
      #Generate some new images
      gen_images = self.generator(tf.random.normal((128,128,1)), training=True)
      #Create the predicted labels, run generator images thru discriminator
      predicted_labels = self.discriminator(gen_images, training=False)#This will return a 1 if its fake #we dont discriminator to learn here, its just to determine the gen image is real or not.
      #Calculate loss, Trick to training to fake out the discriminator #what it tries to do is to generate images that tries to fakeing#  #here generator loss is rewarded, here generated image are actually real images, so we reward our generator for faking out the discriminator, if the discriminator predicts that a generated image is real
      total_g_loss = self.loss_fn(tf.zeros_like(predicted_labels), predicted_labels)
    #Apply backprob.
    g_gradient = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
    self.g_opt.apply_gradients(zip(g_gradient, self.generator.trainable_variables))#zip to do them both at the same time
    return {"d_loss":total_d_loss, "g_loss":total_g_loss}
#

In [ ]:
#create instance of subclass model
fashgan=FashionGAN(generator, discriminator)


In [ ]:
#Compile the model
fashgan.compile(g_opt, d_opt, loss_fn)

In [ ]:
'Build the call back'
import os
import tensorflow as tf
import tensorflow.keras.preprocessing.image as array_to_img
from tensorflow.keras.callbacks import Callback
os.makedirs('/content/drive/MyDrive/images', exist_ok=True)


In [ ]:
class ModelMonitor(Callback):
  def __init__(self, num_img=3, latent_dim=128):#just the random value u r passing to generate
    self.num_img=num_img
    self.latent_dim=latent_dim
  def on_epoch_end(self, epoch, logs=None):
    #generate random
    random_latent_vectors=tf.random.uniform(shape=(self.num_img, self.latent_dim,1))
    generated_images=self.model.generator(random_latent_vectors)
    generated_images *= 255
    generated_images.numpy()
    for i in range(self.num_img):
      img = array_to_img.array_to_img(generated_images[i])
      img.save(os.path.join('/content/drive/MyDrive/images', f'generated_img{i}_{epoch}.png'))

In [ ]:
'Train the model'
hist=fashgan.fit(ds, epochs=20, callbacks=[ModelMonitor()])

In [ ]:
#Review the performance
#we saved out training in a variable called hist, then we can print it and see its performance
hist.history

In [ ]:
plt.suptitle('loss')
plt.plot(hist.history['d_loss'], label='d_loss')
plt.plot(hist.history['g_loss'], label='g_loss')
plt.legend()
plt.show()

#The performance might enhance if we go and train it for 2000 epocs

In [ ]:
'Test the Generator Model'
#before loading the pretrain model, lets test that with baseline model

imgs = generator.predict(tf.random.normal((16,128,1))) # generate 16 images, laten variable 128, then 1)
imgs

'''
Another pretrained model can be downloaded from here :
generatormodel.h5
https://github.com/nicknochnack/GANBasics
and to load its weights:
generator.load_weights('generatormodel.h5')
'''

In [ ]:
fig, axs = plt.subplots(ncols=4,nrows=4, figsize=(20,20)) # changed ax to axs
for idx, img in enumerate(imgs):
    axs.flat[idx].imshow(np.squeeze(img)) # use flat iterator to correctly index the subplots
plt.show()